In [39]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

load_dotenv("../.env")

client = OpenAI(api_key=os.environ['LLM_API_KEY'])

TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

MODEL_NAME = "gpt-4o-mini"

In [40]:
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    
    # 最近1ヶ月をクエリ側で補強する
    augmented = f"{question} 最近1ヶ月"

    response = client.search(augmented)

    # LLMが参照しやすいように、最低限のフィールドだけに整形
    results = []
    for r in response.get("results", []):
        results.append({
            "url": r.get("url"),
            "title": r.get("title"),
            "content": r.get("content"),
            "score": r.get("score"),
        })

    return json.dumps({"result": results})


In [41]:
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]


In [42]:
def ask_question(messages, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    return response

In [ ]:
def handle_tool_call(messages, question):
    # tool_callsが複数ある場合にも対応
    assistant_message = response.choices[0].message
    tool_calls = assistant_message.tool_calls or []
    messages.append(assistant_message)

    for tool in tool_calls:
        function_name = tool.function.name
        arguments = json.loads(tool.function.arguments)
    
        function_response = globals()[function_name](**arguments)

        messages.append({
            "tool_call_id": tool.id,
            "role": "tool",
            "content": function_response,
        })

    return messages

In [ ]:
def process_response(messages, tools, max_steps: int = 5):
    if not messages or messages[0].get("role") != "system":
        messages.insert(0, {
            "role": "system",
            "content": (
                "あなたはWeb検索ツールを使えるアシスタントです。"
                "Web検索ツールの結果に含まれるURL以外は、回答中にURLとして提示してはいけません。"
                "回答では参照したURLを箇条書きで必ず示してください"
                "直近の情報が必要そうな質問では、ツールを積極的に使ってください"
            )
        })

    # 無限ループ防止にLLM問い合わせの上限を設定してループさせる
    for _ in range(max_steps):
        response = ask_question(messages, tools)
        choice = response.choices[0]

        if response.choices[0].finish_reason == 'tool_calls':
            final_response = handle_tool_call(response, question)
            continue
        
        content = (choice.message.content or "").strip()
        messages.append({
            "role": "assistant",
            "content": content,
        })
        return content
    
    fallback = "ツール呼び出しが続いたため、処理を中断します。もう少し具体的な質問にしてください。"
    messages.append({
        "role": "assistant",
        "content": fallback
    })
    return fallback

In [45]:
tools = define_tools()
messages = []

while(True):
    question = input("メッセージを入力:")
    if question.strip()=="":
        break
    display(f"質問:{question}")

    messages.append({"role": "user", "content": question.strip()})
    response_message = process_response(messages, tools)

    print(response_message, flush=True)

print("\n---ご利用ありがとうございました！---")


------define_tools(ツール定義)------


'質問:東京で今後開催される祭りを教えて'

東京で今後開催される祭りについての情報は以下の通りです。

1. **秋祭り in 東京都台東区**  
   - 日付: 2023年9月15日（金）～17日（日）  
   - 内容: 和の文化を楽しめる祭りです。  
   - 参考: [Link](https://trip.iko-yo.net/articles/1529)

2. **神楽坂けんか祭り**  
   - 日付: 2023年10月1日（日）  
   - 内容: 神楽坂で行われる伝統的なお祭り。火を使ったパフォーマンスがあります。  
   - 参考: [Link](https://travel.navitime.com/ja/area/jp/guide/NTJmat1160/)

3. **秋の東京ナイトマーケット**  
   - 日付: 2023年10月中旬  
   - 内容: 食べ物やアートが楽しめるナイトマーケット。詳細は近々発表。  
   - 参考: [Link](https://www.enjoytokyo.jp/feature/matsuri_autumn/cal_oct_tokyo/)

4. **アークヒルズ秋祭り**  
   - 日付: 2023年10月14日（土）～15日（日）  
   - 内容: 地元の特産物を使った飲食ブースが出展されます。  
   - 参考: [Link](https://www.tokyofes.info/ad2023/)

これらの情報をもとに、さらに詳細な情報が必要な場合は、報道や公式サイトをご覧いただくと良いでしょう。


'質問:その中でおすすめの祭りは？'

おすすめの祭りは「秋祭り in 東京都台東区」です。以下にその理由を述べます。

- **文化体験**: 秋祭りでは、伝統的な和の文化を体験でき、地元の特産品や手作りの工芸品などが楽しめます。
- **雰囲気**: 台東区は歴史あるエリアで、祭りの雰囲気を満喫しながら散策するのに適した場所です。
- **家族向け**: 家族連れでも楽しめるアクティビティが豊富で、子供向けのゲームやイベントも行われていることが多いです。

これらの点から、特に秋祭りは他の祭りに比べて参加しやすく、楽しめる要素が多いため、おすすめです。

参考: [秋祭り in 東京都台東区 - trip.iko-yo.net](https://trip.iko-yo.net/articles/1529)

---ご利用ありがとうございました！---
